In [1]:
import numpy as np

# Optimization as ML


La idea es que vamos a  buscar una función que depende de $\Theta$ (conjunto de todos los parámetros) de la forma $J(\Theta)$ que se conoce como función objetivo. De esta forma, estamos interesados en buscar un $$\Theta^*\equiv argmin{J(\Theta)}$$
Se pueden escribir infinits funciones objetivas, no onstante, es muy fácil utilizar una función de pérdida $\mathcal{L}$, es decir que podemos escribir:

$$J(\Theta)=\frac{1}{n}\sum_{i=1}^{n}\mathcal{l}(h(x^{(i)};\Theta),y^{(i)})+\lambda R(\Theta)$$

Donde $R(\Theta)$ se le conoce como $\textbf{Regularizador}$ para evitar el overfitting, $\lambda$ es un hiperparámetro positivo de dicta que tan compleja debería ser la hipótesis $h(x^{(i)};\Theta)$.

Una vez tenemos esta función con una hipótesis, se pueden desarrollar $\textit{algos}$ que permitan optimizarla para encontrar el $\Theta^*$.

## Linear Logistic Classifiers

También se les conoce como $\textbf{Logistic Regression}$ que nos permite establecer un nuevo tipo de función de pérdida y de funciones de clase. La función de predicción toma la forma:

$$LLC(x;\theta,\theta_0)=\sigma(\theta^Tx+\theta_0)$$

Para esto debemos recordar la función sigmoide:

$$\sigma(z)=\frac{1}{1+e^{-z}}$$

A este valor que arroja $\sigma(z) \in (0,1)$ le podemos interpretar como la posibilidad de que $z$ sea positivo.

De esta forma, si queremos un clasificador, tenemos que poner un límite en esta función, por ejemplo la más simple puede ser 
$$\sigma(\theta^Tx+\theta_0)\ge 0.5\iff \theta^Tx+\theta_0\ge0$$.
$$ $$
Procedemos entonces a definir las funciones de pérdida y regularizador.
* La función de pérdida está relacionada inversamente con la probabilidad de que $\Theta$ de cuenta de los datos, para ello hacemos las siguientes definiciones:
$$g^{(i)}\equiv\sigma(\theta^Tx^{(i)}+\theta_0)$$
De esta forma definimos una función de pérdida logarítmica de la forma
$$\mathcal{L}_{nll}=-\sum _{i = 1}^ n \left( {y^{(i)}}\log {g^{(i)}} + (1 - y^{(i)})\log (1 - g^{(i)})\right)\; \; $$

$$\mathcal{L}_\text {nll}(\text {guess},\text {actual}) = -\left(\text {actual}\cdot \log (\text {guess}) + (1 - \text {actual})\cdot \log (1 - \text {guess})\right) \; \; $$

* Una forma sencilla de establecer un estabilizador que evite el overfitting es: 
$$R(\theta)=||\Theta-\Theta_{prior}||\equiv||\theta||$$
$$\;$$

Ya tenemos casi todo listo, nos falta buscar una forma sencilla pero útil de hacer una optimización con los datos y la función de pérdida, para esto vamos a introducir:


## Gradient Descent

Es por mucho uno de los algoritmos más eficientes y rápidos de entender para realizar optimización, vamos entonces a analizarlo:
* $\textbf{Una dimensión:}$
Partimos de un valor de $\theta$, Vamos a ir calculando derivadas y a ir haciendo pequeños pasos en dirección opuesta.


In [2]:
def GradDesUniDim(th,f,derivf,epsi,eta,t_max=1000):
    """"Pasar f como función lambda"""
    ths=[th];ban=True;t=0
    while ban:
        t+=1
        ths.append(ths[t-1]-eta*derif(ths[t-1]))
        if (f(ths[t])-f(ths[t-1]))<epsi or t>=t_max: ban=False
    return ths[t]

* $\textbf{Múltiples dimesiones:}$ Salimos con una función que tiene $m$ argumentos, es decir que $\theta$  es de tamaño $m\times 1$ y supongamos que $f(\Theta)$ es una función escalar. También se necesitan $\nabla_{\theta} f$ y una primera aproximación para $\theta$


In [ ]:
def gd(f, df, x0, step_size_fn, max_iter):
    prev_x = x0
    fs = []; xs = []
    for i in range(max_iter):
        prev_f, prev_grad = f(prev_x), df(prev_x)
        fs.append(prev_f); xs.append(prev_x)
        if i == max_iter-1:
            return prev_x, fs, xs
        step = step_size_fn(i)
        prev_x = prev_x - step * prev_grad

Para realizar la función de gradiente numérico, si $x$ es un vector columna con los parámetros de la función escalar $f$:

In [ ]:
def num_grad(f, delta=0.001):
    def df(x):
        g = np.zeros(x.shape)
        for i in range(x.shape[0]):
            xi = x[i,0]
            x[i,0] = xi - delta
            fxm = f(x)
            x[i,0] = xi + delta
            fxp = f(x)
            x[i,0] = xi
            g[i,0] = (fxp - fxm)/(2*delta)
        return g
    return df

Entonces una función que minimice completamente es:


In [3]:
def minimize(f, x0, step_size_fn, max_iter):
    df = num_grad(f)
    return gd(f, df, x0, step_size_fn, max_iter)

## Otra Función de Pérdida: Linear Support Vector Machines

Existe una forma un poco más eficiente de tomar las funciónes de pérdida con respecto a una $\gamma$ de referencia, a esta función se le llama Hingle Loss:
$$L_{H}\left( \gamma ,\gamma _{ref}\right) =\begin{cases}1-\left( \dfrac{\gamma}{\gamma_{ref}}\right) ;\gamma  <\gamma _{ref}\\
0;otro \end{cases}$$

De esta forma, en resumen podemos concluir que por SMV la función objetivo de la optimización es : 

$$J(\theta,\theta_0)=\frac{1}{n}\sum_{i=1}^{n}L_H\big(y^{(i)}\cdot(\theta x^{(i)}+\theta_0)\big)+\lambda ||\theta||^2$$

## Implementación de $gd$ a SVM

En esta implementación, $x$ es $d\times n$, $y$ es $1\times n$, $th$ es $1\times 1$ y $lam$ es un escalar

In [ ]:
def hinge(v):
    return np.where(v >= 1, 0, 1 - v)

def hinge_loss(x, y, th, th0):
    return hinge(y * (np.dot(th.T, x) + th0))

def svm_obj(X, y, th, th0, lam):
    return np.mean(hinge_loss(X, y, th, th0)) + lam * np.linalg.norm(th) ** 2

In [ ]:
def d_hinge(v):
    return np.where(v >= 1, 0, -1)
def d_hinge_loss_th(x, y, th, th0):
    return d_hinge(y*(np.dot(th.T, x) + th0))* y * x
def d_hinge_loss_th0(x, y, th, th0):
    return d_hinge(y*(np.dot(th.T, x) + th0)) * y
def d_svm_obj_th(x, y, th, th0, lam):
    return np.mean(d_hinge_loss_th(x, y, th, th0), axis = 1, keepdims = True) + lam * 2 * th
def d_svm_obj_th0(x, y, th, th0, lam):
    return np.mean(d_hinge_loss_th0(x, y, th, th0), axis = 1, keepdims = True)
def svm_obj_grad(X, y, th, th0, lam):
    grad_th = d_svm_obj_th(X, y, th, th0, lam)
    grad_th0 = d_svm_obj_th0(X, y, th, th0, lam)
    return np.vstack([grad_th, grad_th0])

In [ ]:
def batch_svm_min(data, labels, lam):
    def svm_min_step_size_fn(i):
       return 2/(i+1)**0.5
    init = np.zeros((data.shape[0] + 1, 1))

    def f(th):
      return svm_obj(data, labels, th[:-1, :], th[-1:,:], lam)

    def df(th):
      return svm_obj_grad(data, labels, th[:-1, :], th[-1:,:], lam)

    x, fs, xs = gd(f, df, init, svm_min_step_size_fn, 10)
    return x, fs, xs